# Task 1: Evaluating Retrieval Quality

## Imports

In [1]:
from nltk.stem import WordNetLemmatizer
import contractions
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

## Implement BM25 code from Coursework 1
NOTE: none of this code is new. It is all reused from the previous coursework, and adapted to the validation dataset.

Here we implement the BM25 code from Coursework 1, with the same text normalisation function, and all, but we do it on the validation data instesd of the candidate passages top1000 data.
From this we will obtain a file where for each query we have the top 100 (max) passages ranked from highest BM25 score to lowest. 
Then we use the ranking and the relevance scores from the validation set to calculate the performance metrics for the BM25 retrieval model.

This is the nature of the other tasks - build a retrieval model (here we already have it from the previous coursework, but in the other tasks we need to make it and train it with the training set), and then once we have the retrieval model, we feed the validation data to it, obtain the sorted (qid,pid) pairs in ranking order with their respective scores (whatever score each model uses), and then we use that ranking alongside the relevance scores in the validation set to calculate the performance metrics of the retrieval model:  mAP and NDCG scores.

In [2]:
def normalise(text):
    '''
    Function that normalises text and returns tokens.
    Input: text --> text string we want to tokenise
    Output: tokens --> list of tokens taken from the text string
    '''

    text = text.lower() # convert all to lower case
    text = contractions.fix(text) # expand contractions
    text = text.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))) # remove punctuation
    tokens = text.split() # tokenisation
    filtered_tokens = [w for w in tokens if not w in stop_words] # remove stop words
    filtered_tokens = list(map(lemmatizer.lemmatize, filtered_tokens)) # lemmatization of nouns

    return filtered_tokens

In [3]:
cp = pd.read_csv('validation_data.tsv', delimiter='\t')

In [4]:
relevancies = cp[['qid', 'pid', 'relevancy']].copy()

In [5]:
passages = cp[['pid', 'passage']].copy()
passages = passages.drop_duplicates()
passages = passages.reset_index(drop=True)

In [6]:
tq = cp[['qid', 'queries']].copy()
tq = tq.drop_duplicates()
tq = tq.reset_index(drop=True)

In [7]:
inv_index = {}

def inverted_index_func(row):
    pid = row['pid']
    passage = row['passage']
    check = normalise(passage)
    unique_words = list(set(check))

    for item in unique_words:
        if item not in inv_index:
            inv_index[item] = {}
        if item in inv_index:
            inv_index[item][pid] = check.count(item) # Frequency of the word (not normalised)

_ = passages.apply(lambda row: inverted_index_func(row), axis=1)

In [8]:
# TF·IDF passages
p_tfidf = {i:{} for i in passages['pid']}

# Also calculate the length of passages which will be useful in BM25
len_passages = {}
N = len(passages)

def p_tfidf_func(row):
    pid = row['pid']
    passage = row['passage']
    check = normalise(passage)
    len_passages[pid] = len(check)
    unique_words = list(set(check))

    for item in unique_words:
        tf = inv_index[item][pid] / len(check) # Normalise the frequency from inverted index
        idf = np.log(N/len(inv_index[item]))
        p_tfidf[pid][item] = tf*idf
    p_tfidf[pid]['norm_of_passage'] = np.linalg.norm(np.array(list(map(p_tfidf[pid].get, p_tfidf[pid].keys())))) # add norm of the passage

_ = passages.apply(lambda row: p_tfidf_func(row), axis=1)

In [9]:
# For simplicity, and keeping things separate, we compute the frequencies of the query terms separately
qf_dict = {i:{} for i in tq['qid']}

def qf_func(row):
    qid = row['qid']
    query = row['queries']
    check = normalise(query)
    unique_words = list(set(check))

    for item in unique_words:
        if item in inv_index.keys():
            qf_dict[qid][item] = check.count(item) # non-normalised frequency

_ = tq.apply(lambda row: qf_func(row), axis=1)

In [10]:
def BM25(n, N, k1, k2, b, dl, avdl, f, qf, r=0, R=0):
    '''
    BM25 score calculating function

    Inputs
    n: (vector of integers) number of total docs. containing each term in the query (each vector element corresponds to a term)
    N: (integer) total number of documents we have
    k1: (scalar) constant parameter set empirically
    k2: (scalar) constant parameter set empirically
    b: (scalar) constant parameter set empirically
    dl: (integer) document length --> number of tokens
    avdl: (scalar) average document length --> average number of tokens in the set of documents
    f: (vector of integers) frequency in the document of each term in the query (each vector element corresponds to a term)
    qf: (vector of integers) frequency in the document of each term in the query (each vector element corresponds to a term)
    r: (vector of integers) number of relevant docs. containing each term in the query (each vector element corresponds to a term)
    R: (integer) total number of relevant documents

    Note that if we do not have information about relevance feedback, r and R are set to 0

    Output
    score: (scalar) BM25 score of a document with respect to a query

    '''

    K = k1 * ((1 - b) + b * (dl/avdl))

    score = np.sum( np.log( ((r+0.5)/(R-r+0.5)) / ((n-r+0.5)/(N-n-R+r*0.5)) ) * (((k1+1)*f)/(K+f)) * (((k2+1)*qf)/(k2+qf)) )

    return score

In [11]:
N = len(passages)
k1 = 1.2
k2 = 100
b = 0.75
avdl = sum(len_passages.values()) / len(len_passages)

BM25_scores = np.array([[0,0,0]])

for k in range(len(tq)):
    scores = []
    qid = tq['qid'][k]
    query = qf_dict[qid]
    query_words = query.keys()

    for pid in cp.loc[cp['qid'] == tq['qid'][k]]['pid']:
        
        passage = p_tfidf[pid]
        passage_words = passage.keys()

        common = list(set(query_words) & set(passage_words))

        dl = len_passages[pid]

        n, f, qf = np.zeros(len(common)), np.zeros(len(common)), np.zeros(len(common))
        for i in range(len(common)):
            n[i] = len(inv_index[common[i]])
            f[i] = inv_index[common[i]][pid]
            qf[i] = qf_dict[qid][common[i]]

        score = BM25(n,N,k1,k2,b,dl,avdl,f,qf)

        scores.append([qid,pid,score])
    
    scores = np.array(scores, dtype="O")
    scores = scores[np.argsort(-scores[:,-1])] # sort in descending order
    
    BM25_scores = np.append(BM25_scores, scores[:100,:], axis=0)

BM25_scores = BM25_scores[1:,:] # remove the [0,0,0] row we used to initialise

In [12]:
pd.DataFrame(BM25_scores).to_csv("bm25_cw2.csv", header=None, index=None)

## Define functions to calculate mAP and NDCG
Note that our rankings are in a .csv file where we have (qid, pid, score), and the (qid, pid) pairs are in order from best to worse, for each qid.

In [13]:
# First we load the rankings as a data frame
ranking = pd.read_csv('bm25_cw2.csv', delimiter=',', header=None, names=['qid','pid','score'])

In [14]:
# mAP function
def AP(queries, ranking, relevancies, k):
    '''
    Function that computes the Average Precision (AP) metric for each query in 'queries', based on a ranking determined by 
    a retrieval model, where queries are matched with passages from most relevant to least relevant, and based on relevancies 
    between queries and passages.

    Inputs:
    queries = data frame of queries for which you want to calculate the AP metric (contains qid and actual query)
    ranking = data frame of queries and passages pairs, where higher score pairs are ranked higher (for each query)
    relevancies = data frame of relevancies between each possible (qid,pid) pair
    k = top k passages you want to take into account when calculating the AP metric

    Outputs:
    APs = list of AP@k metric for each query, in the same order of appearance as the input list 'queries'
    mAP = mean Average Precision of all the queries
    '''

    APs = []

    for q in queries['qid']:
        AP_values = []
        cum_rel = 0 # cumulative number of relevant passages found in the ranking

        max_k = len(ranking[ranking['qid'] == q])
        iter = min(k,max_k) # This is because we some queries do not have that many candidate passages

        for i in range(1,iter+1):
            p = int(ranking[ranking['qid'] == q].reset_index(drop=True).iloc[i-1]['pid'])
            relevancy = relevancies[(relevancies['qid'] == q) & (relevancies['pid'] == p)]['relevancy'].values.item()
            if relevancy != 0: # we operate when we encounter a relevant passage
                cum_rel += relevancy
                AP_values.append(cum_rel / i) 

        if len(AP_values) != 0:    
            APs.append(sum(AP_values)/len(AP_values))
        else: # we do this to avoid the computing error of dividing 0/0
            APs.append(0)

    mAP = np.mean(APs)

    return APs, mAP

In [15]:
AP_3, mAP_3 = AP(tq, ranking, relevancies, 3)
AP_10, mAP_10 = AP(tq, ranking, relevancies, 10)
AP_100, mAP_100 = AP(tq, ranking, relevancies, 100)

In [19]:
print(mAP_3, mAP_10, mAP_100)

0.1923635307781649 0.22908750898733476 0.24050810167578454


In [16]:
# NDCG function
def NDCG(queries, ranking, relevancies, k):
    '''
    Function that computes the Normalized Discounted Cumulative Gain (NDCG) metric for each query in 'queries', 
    based on a ranking determined by a retrieval model, where queries are matched with passages from most relevant 
    to least relevant, and based on relevancies between queries and passages.

    Inputs:
    queries = data frame of queries for which you want to calculate the AP metric (contains qid and actual query)
    ranking = data frame of queries and passages pairs, where higher score pairs are ranked higher (for each query)
    relevancies = data frame of relevancies between each possible (qid,pid) pair
    k = top k passages you want to take into account when calculating the AP metric

    Outputs:
    NDCGs = list of AP@k metric for each query, in the same order of appearance as the input list 'queries'
    mNDCG = mean Average Precision of all the queries
    '''

    NDCGs = []

    for q in queries['qid']:
        DCG = 0
        IDCG = 0

        max_k = len(ranking[ranking['qid'] == q])
        iter = min(k,max_k) # This is because we some queries do not have that many candidate passages

        # Get the relevancies for the ideal ranking in order
        sorted_revs = relevancies[relevancies['qid'] == q].sort_values(by=['relevancy'], ascending=False)['relevancy'].values

        for i in range(1,iter+1):
            IDCG += (2**sorted_revs[i-1] - 1)/np.log2(i+1)

            p = int(ranking[ranking['qid'] == q].reset_index(drop=True).iloc[i-1]['pid'])
            rel = relevancies[(relevancies['qid'] == q) & (relevancies['pid'] == p)]['relevancy'].values.item()
            DCG += (2**rel - 1)/np.log2(i+1)

        if IDCG != 0:
            NDCGs.append(DCG/IDCG)
        else: # we do this to avoid the computing error of dividing 0/0
            NDCGs.append(0)

    mNDCG = np.mean(NDCGs)

    return NDCGs, mNDCG

In [17]:
NDCG_3, mNDCG_3 = NDCG(tq, ranking, relevancies, 3)
NDCG_10, mNDCG_10 = NDCG(tq, ranking, relevancies, 10)
NDCG_100, mNDCG_100 = NDCG(tq, ranking, relevancies, 100)

In [18]:
print(mNDCG_3, mNDCG_10, mNDCG_100)

0.2111449285596187 0.2870114730902176 0.3571446023375422


# End of Task 1